In [8]:
import json
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import numpy as np
import inflect
import time
import gensim
engine = inflect.engine()

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
recipes = json.load(open('../generated/all_recipes_repr.json'))

In [4]:
len(recipes)

963458

In [8]:
def singularize(word):
    ingr = engine.singular_noun(word)
    return word if (not ingr) else ingr

def clean_ing_word(word) : return singularize(word).lower()

def clean_whole_ing(ing) : return " ".join([clean_ing_word(word) for word in ing.split(" ")])

Word2Vec training

In [10]:
start_time = time.time()
model = gensim.models.Word2Vec(recipes, min_count=2)

time_after_creation = time.time()
print("creation of model :", time_after_creation - start_time, "sec.")

model.train(recipes, total_examples=len(recipes), epochs=10)

time_after_training = time.time()
print("model training :", time_after_training - time_after_creation, "sec.")

model.save('food_embeddings.bin')

time_after_saving = time.time()
print("model saving :", time_after_saving - time_after_training, "sec.")

creation of model : 11.016540765762329 sec.
model training : 20.05038547515869 sec.
model saving : 0.01994633674621582 sec.


Results investigation

In [13]:
model.wv.most_similar("lamb")

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('lean lamb', 0.8490262031555176),
 ('lamb shoulder', 0.7714232802391052),
 ('shoulder lamb chop', 0.6138100624084473),
 ('loin lamb', 0.6116103529930115),
 ('lamb chop', 0.6047450304031372),
 ('lamb rib chop', 0.5806523561477661),
 ('lamb shank', 0.5717580318450928),
 ('eggplant', 0.5647945404052734),
 ('mutton', 0.563430905342102),
 ('goat meat', 0.5388809442520142)]

We see that the adjectives are creating new ingredients. Given we are only interested in flavours associations, we should ignore them. Indeed, shredded mushrooms and dried mushrooms should have the same flavour. 

In [24]:
model.wv.similarity(w1="salt", w2="pepper")

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


-0.47766

In [ ]:
 model.most_similar(positive=['rice', 'tomato'], negative=['pasta'], topn=1)